In [1]:
import database_faiss as faissdb
import database_sql as db
import openai
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [2]:
model = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE")
embedding_dim = len(model.embed_query("hello world"))
import faiss
index = faiss.IndexFlatL2(embedding_dim)
vector_store = faissdb.connect_faiss(embedding_model=model, index=index, faiss_index_dir='../01_data/project_faiss')

In [3]:
total_vectores = vector_store.index.ntotal
print("Número de vectores indexados:", total_vectores)

Número de vectores indexados: 79044


In [4]:
dimension = vector_store.index.d
print("Dimensionalidad de los vectores:", dimension)

Dimensionalidad de los vectores: 768


In [5]:
query = "¿Cuántas horas de prácticas curriculares equivalen a un crédito ECTS?"

In [6]:
results = vector_store.similarity_search_with_score(query, k=3)
contexto = results[0][0].page_content
id = results[0][0].id
results

[(Document(id='40722', metadata={'page': 4}, page_content='En resumen,  el reconocimiento  de la labor \ndocente (denominada  CREP) para 1 crédito ECTS, \nestará comprendido  entre 11 y 12 horas. De tal \nforma que 1 crédito ECTS contiene 10 horas de \nactividad docente presencial,  y las 2 horas \nrestantes,  como máximo, corresponden  al \nreconocimiento  de actividades.  \n 2. Se mantiene  el reconocimiento  de la labor \ndocente del profesorado  de tal forma\n que se \nasume que 1 crédito ECTS contiene 10 horas de \nactividad docente presencial.'),
  0.7500472),
 (Document(id='16507', metadata={'page': 2}, page_content='titulaciones  oficiales, serán contempladas  en el \nsuplemento  europeo al título (SET) conforme  \ndetermine  la normativa  vigente. \n \nTercero.‐ La redacción  de los apartados  1 y 2 del \nartículo 5 queda de la siguiente manera: \n 1. La duración de las prácticas académicas  externas \nserá la siguiente,  considerando  que 1 crédito ECTS \nequivale a 25 horas 

In [7]:
def get_response(client, query, context):
    response = client.chat.completions.create(
        model="poligpt",
        messages = [
            {
                "role": "user",
                "content": f"Responde a la siguiente pregunta usando el contexto suministrado.\n\nPregunta: {query}\nContexto: {context}"
            }
        ]
    )
    return response.choices[0].message.content

In [8]:
client = openai.OpenAI(
        api_key="sk-1C-hWjmHEW05iQjwmr9EnA",
        base_url="https://api.poligpt.upv.es"
    )

In [9]:
answer = get_response(client, query, contexto)
print("Response:", answer)

Response: Según el contexto, 1 crédito ECTS equivale a entre 11 y 12 horas de prácticas curriculares en total, considerando tanto la actividad docente presencial como el reconocimiento de actividades adicionales.

Desglosado:

- Actividad docente presencial: 10 horas
- Reconocimiento de actividades adicionales: hasta 2 horas

Por lo tanto, un crédito ECTS puede equivaler a entre 11 y 12 horas en total.
